In [1]:
import os
import datetime as dt
import requests
from bs4 import BeautifulSoup
from tqdm.auto import tqdm 
from dotenv import load_dotenv
import pymongo
load_dotenv(verbose=True)

/home/test/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
IP = os.getenv("IP")
PORT = os.getenv("PORT")
DATABASE = os.getenv("DATABASE")
COL = os.getenv("COLLECTION")

In [5]:
conn = pymongo.MongoClient("mongodb://{}:{}".format(IP,PORT))
db = conn[DATABASE]
col = db[COL]

In [6]:
total = []
exception = []

for i in tqdm(range(64,0,-1)): 
    url = f"https://munjang.or.kr/board.es?mid=a20103000000&bid=0003&nPage={i}"
    plain_text = requests.get(url).text
    contents = BeautifulSoup(plain_text,'html.parser')
    for c in contents.find_all("li", {"class" : "list_li"}):
        now = dt.datetime.now()
        global novels
        novels = {}
        try: 
            info = c.find('strong').text.split(" - ")
            novels['title'] = info[1]
            novels["author"] = info[0]
            novels["links"] = "https://munjang.or.kr" + c.find("a")["href"]
            novels["crawling_date"] = now.strftime("%Y-%m-%d %H:%M:%S")
            total.append(novels)
            col.insert_one(novels)
        except:
            exception.extend(info)


100%|██████████| 64/64 [00:17<00:00,  3.75it/s]


In [8]:
db.list_collection_names()

['meta']